# Getting Bsky Data
Sara Rosenau 2/21/2025  
This notebook will feature me trying to learn the Bsky API and collect some data from it

## Playing around with the api a bit
- First, import the api (`atproto`) and start a new client session

In [1]:
from atproto import Client, client_utils

In [10]:
fname = r'private\pw.txt'
file = open(fname, 'r')
pw = file.read()
file.close()

In [ ]:
client = Client()
client.login('sararosenauling.bsky.social', pw)

### Making a post from the api!

In [4]:
#client.send_post(text='testing api')

### Getting posts from my timeline
- Following [this example](https://github.com/MarshalX/atproto/blob/main/examples/home_timeline.py)
- Getting 10 posts from my following feed

In [5]:
def main() -> None:
    print('Home (Following):\n')

    timeline = client.get_timeline(limit=10, algorithm='reverse-chronological')
    for feed_view in timeline.feed:
        action = 'New Post'
        if feed_view.reason:
            action_by = feed_view.reason.by.handle
            action = f'Reposted by @{action_by}'

        post = feed_view.post.record
        author = feed_view.post.author

        print(f'[{action}] {author.display_name}: {post.text}')

In [6]:
main()

Home (Following):

[New Post] Joshua J. Friedman: What’s your source for this? cc: @mollyknight.bsky.social
[New Post] Quinn Dombrowski: Today is the 3rd anniversary of the full-scale invasion, but also the 1st anniversary of the day my now-girlfriend messaged me back, setting off a chain of events that became two scarves, and an opportunity to make a real and different choice about what I wanted my life to be like.
[New Post] The Guillotine Shouter: Not to get political but where the fuck did I put my sunglasses
[Reposted by @banalplay.bsky.social] Jay Willis: Odd how the same companies asserting strong First Amendment rights in the election spending context have very little to say when the government threatens them with criminal charges over milquetoast corporate diversity initiatives ballsandstrikes.org/law-politics...
[New Post] Katie Tightpussy, Editor-In-Chief: unassisted. words
[New Post] Nina Markl 🏳️‍⚧️🏳️‍🌈: some of them! not all of them i assume?
[New Post] Scoob: That's a pr

### Trying a search
- just trying to get a hang of what is going on with the `client.app.bsky.feed.search_posts()`
  - turns out that it returns a Response type object and the api is built on something called "pydantic"
  - `object.model_dump()` returns a dictionary with a list of posts and then a dictionary for each post containing a lot of metadata
  - with a for loop and some indexing we can get all the info we need!

In [44]:
import requests

In [11]:
def print_search(query):
    print(query, 'Search Results:\n')
    results = client.app.bsky.feed.search_posts({'q': query, 'limit': 10, 'sort': 'top'})
    results_dict = results.model_dump()
    for post in results_dict['posts']:
        text = post['record']['text']
        author = post['author']['handle']
        print(author+':', text+'\n')

In [12]:
print_search('unalive')

unalive Search Results:

maiamindel.bsky.social: "Orange Man Bad" and "Unalive the Boer" are universal truths that all living beings are innately attuned to. Like a message from the Creator

baratiddys.bsky.social: postponing my unalive

kingserpentico.bsky.social: I *AM* on vacation and people still wanna unalive me at work.

aliothfox.ursamajorartworks.com: Y'all, please stop saying "unalive" and "self-delete" and so forth. There's no algorithm to fight here and a lot of people have the word "suicide" muted for a reason. The euphemisms are neither cute nor helpful.

vitaminpac1.bsky.social: Why did I marry someone who picks the WORST FUCKING MOVIES

My god I either want to die of boredom, want to unalive myself, or am too confused to decide

nacl.sh: unalive journal

usakoing.bsky.social: Chat if I unalive myself

verynormalguy.bsky.social: wait, people actually say unalive? we’re all so cooked

sixfootfiveguy.com: An app for physician-assisted unalive: 

Snuffies

killjill.itch.io: 

## Getting some actual data
- Building a function called `search2df()` that is pretty similar to the `print_search()` function above
    - makes a list of dictionaries for each post containing the metadata I might want for each search term
    - then converts it into a pandas DataFrame!

In [51]:
import pandas as pd
def search2df(query):
    results = client.app.bsky.feed.search_posts({'q': query, 'limit': 10, 'sort': 'top'})
    results_dict = results.model_dump()
    query_data = []
    for post in results_dict['posts']:
        metadata = {}
        metadata['text'] = post['record']['text']
        metadata['author'] = post['author']['handle']
        metadata['display_name'] = post['author']['display_name']
        metadata['date'] = post['record']['created_at']
        metadata['likes'] = post['like_count']
        metadata['quotes'] = post['quote_count']
        metadata['replies'] = post['reply_count']
        metadata['reposts'] = post['repost_count']
        metadata['uri'] = post['uri']
        metadata['query'] = query
        query_data.append(metadata)
    query_df = pd.DataFrame(query_data)
    return query_df

In [49]:
unalive_top_df = search2df('unalive')

In [50]:
unalive_top_df

,text,author,display_name,date,likes,quotes,replies,reposts,uri,query
0,"""Orange Man Bad"" and ""Unalive the Boer"" are un...",maiamindel.bsky.social,Maia,2025-02-23T17:07:01.256Z,89,1,1,16,at://did:plc:ur77nun2q74loi34r2e6r43u/app.bsky...,unalive
1,postponing my unalive,baratiddys.bsky.social,Luis 🤍,2025-02-20T16:47:35.943Z,21,0,1,1,at://did:plc:dluxclbmnsh3bt6wyih5l6ds/app.bsky...,unalive
2,I *AM* on vacation and people still wanna unal...,kingserpentico.bsky.social,SNAKEMAN,2025-02-20T02:26:55.167Z,61,0,3,1,at://did:plc:pv4626x5y7zxxsupflujztqb/app.bsky...,unalive
3,"Y'all, please stop saying ""unalive"" and ""self-...",aliothfox.ursamajorartworks.com,Alioth Daddyfox,2025-02-21T17:55:36.580Z,114,2,7,28,at://did:plc:5mkojgjmjfhdpd5lvepg2q6h/app.bsky...,unalive
4,Why did I marry someone who picks the WORST FU...,vitaminpac1.bsky.social,Vitamin Bee 🐝,2025-02-24T01:18:42.171Z,21,0,7,1,at://did:plc:37nxbbnnrozlllwyfngsalya/app.bsky...,unalive
5,unalive journal,nacl.sh,Salty 🇵🇸 🇵🇸,2025-02-23T03:57:47.089Z,28,0,2,1,at://did:plc:i5mvo22kcxsmqn6e3lgjbfnf/app.bsky...,unalive
6,Chat if I unalive myself,usakoing.bsky.social,serena✿,2025-02-22T16:12:41.198Z,4,4,3,0,at://did:plc:g66g533526vxkekvvxugkcah/app.bsky...,unalive
7,"wait, people actually say unalive? we’re all s...",verynormalguy.bsky.social,evan n,2025-02-20T14:42:20.392Z,12,2,0,0,at://did:plc:fcfkcslvkunsrcldbzmx2imc/app.bsky...,unalive
8,An app for physician-assisted unalive: \n\nSnu...,sixfootfiveguy.com,Joe Spaceman,2025-02-19T16:11:55.170Z,19,0,0,0,at://did:plc:dpxvpwbgnumoxvegabzrpqg7/app.bsky...,unalive
9,I watched something and it used the word 'unal...,killjill.itch.io,JILLIAN F. KILLS,2025-02-23T19:07:48.841Z,14,0,1,0,at://did:plc:3lcmedxtqgg72d4ag57go72w/app.bsky...,unalive
